In [1]:
import scanpy as sc
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import anndata
import sc_utils
import scvi

Global seed set to 0
/projects/b1038/Pulmonary/nmarkov/2022-multimodal/code/venv/lib/python3.9/site-packages/pytorch_lightning/utilities/warnings.py:53: LightningDeprecationWarning: pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6 and will be removed in v1.8. Use the equivalent function from the pytorch_lightning.utilities.rank_zero module instead.
  new_rank_zero_deprecation(
/projects/b1038/Pulmonary/nmarkov/2022-multimodal/code/venv/lib/python3.9/site-packages/pytorch_lightning/utilities/warnings.py:58: LightningDeprecationWarning: The `pytorch_lightning.loggers.base.rank_zero_experiment` is deprecated in v1.7 and will be removed in v1.9. Please use `pytorch_lightning.loggers.logger.rank_zero_experiment` instead.
  return new_rank_zero_deprecation(*args, **kwargs)


In [2]:
import h5py
import hdf5plugin

In [3]:
f = h5py.File("../data/train_multi_inputs.h5")

In [4]:
names = f["train_multi_inputs"]["axis0"].asstr()[:]

In [5]:
names = pd.Series(names)

In [6]:
chromosomes = pd.Series(names.str.split(":").str[0].unique())

In [7]:
chromosomes = chromosomes[chromosomes.str.startswith("chr")]

In [8]:
from typing import Iterator, Mapping, Tuple, NamedTuple, Sequence, Union

from absl import app
from absl import flags
from absl import logging
import haiku as hk
import jax
import jax.numpy as jnp
import numpy as np
import optax
import matplotlib.pyplot as plt
import math

In [16]:
batch_size = 128 #"Size of the batch to train on.")
learning_rate = 0.001 # "Learning rate for the optimizer.")
training_steps = 1000 #, "Number of training steps to run.")
eval_frequency = 10 #, "How often to evaluate the model.")
random_seed = 42 #, "Random seed.")
l1_coef = 0.5

In [10]:
PRNGKey = jnp.ndarray
Batch = Mapping[str, np.ndarray]
HiddenSize = Union[int, tuple]

MNIST_IMAGE_SHAPE: Sequence[int] = (28, 28, 1)

In [11]:
class Encoder(hk.Module):
    """Encoder model."""

    def __init__(self, hidden_size: HiddenSize = 512, latent_size: int = 10):
        super().__init__()
        if not isinstance(hidden_size, tuple):
            hidden_size = (hidden_size,)
        self._hidden_size = hidden_size
        self._latent_size = latent_size

    def __call__(self, x: jnp.ndarray) -> Tuple[jnp.ndarray, jnp.ndarray]:
        x = hk.Flatten()(x)
        activation_sum = 0
        for layer_size in self._hidden_size:
            x = hk.Linear(layer_size)(x)
            activation_sum += jnp.mean(jnp.abs(x))
            x = jax.nn.leaky_relu(x)

        return hk.Linear(self._latent_size)(x), activation_sum


class Decoder(hk.Module):
    """Decoder model."""

    def __init__(
        self,
        hidden_size: HiddenSize = 512,
        output_shape: Sequence[int] = MNIST_IMAGE_SHAPE,
    ):
        super().__init__()
        if not isinstance(hidden_size, tuple):
            hidden_size = (hidden_size,)
        self._hidden_size = hidden_size
        self._output_shape = output_shape

    def __call__(self, z: jnp.ndarray) -> jnp.ndarray:
        activation_sum = 0
        for layer_size in reversed(self._hidden_size):
            z = hk.Linear(layer_size)(z)
            activation_sum += jnp.mean(jnp.abs(z))
            z = jax.nn.leaky_relu(z)

        return hk.Linear(np.prod(self._output_shape))(z), activation_sum


class AEOutput(NamedTuple):
    data: jnp.ndarray
    latent: jnp.ndarray
    act_sum: float


class AutoEncoder(hk.Module):
    """Main VAE model class, uses Encoder & Decoder under the hood."""

    def __init__(
        self,
        hidden_size: HiddenSize = 512,
        latent_size: int = 10,
        output_shape: Sequence[int] = MNIST_IMAGE_SHAPE,
    ):
        super().__init__()
        self._hidden_size = hidden_size
        self._latent_size = latent_size
        self._output_shape = output_shape

    def __call__(self, x: jnp.ndarray) -> AEOutput:
        x = x.astype(jnp.float32)
        z, act_sum_en = Encoder(self._hidden_size, self._latent_size)(x)
        data, act_sum_de = Decoder(self._hidden_size, self._output_shape)(z)
        act_sum_total = act_sum_en + act_sum_de + jnp.mean(jnp.abs(z))
        return AEOutput(data, z, act_sum_total)

In [12]:
@jax.jit
def loss_fn(params: hk.Params, batch) -> jnp.ndarray:
    output: AEOutput = model.apply(params, batch)
    return jnp.mean(jnp.square(batch - output.data)) + l1_coef * output.act_sum

In [13]:
@jax.jit
def update(
    params: hk.Params,
    opt_state: optax.OptState,
    batch: Batch,
) -> Tuple[hk.Params, optax.OptState]:
    """Single SGD update step."""
    grads = jax.grad(loss_fn)(params, batch)
    updates, new_opt_state = optimizer.update(grads, opt_state)
    new_params = optax.apply_updates(params, updates)
    return new_params, new_opt_state

In [ ]:
for c in chromosomes:
    c_names = names.str.startswith(c + ":")
    c_data = f["train_multi_inputs"]["block0_values"][:, c_names.values]
    c_shape = c_names.sum()
    print(f"Training for chromosome {c}: {c_shape}")
    model = hk.transform(
        lambda x: AutoEncoder(
            hidden_size=(c_shape, c_shape // 16),
            latent_size=64, # TODO: try different sizes, including 1
            output_shape=(c_shape,)
        )(x)
    )  # pylint: disable=unnecessary-lambda
    model = hk.without_apply_rng(model)
    optimizer = optax.adam(learning_rate)
    rng = jax.random.PRNGKey(1066)
    params = model.init(rng, np.zeros((c_shape,)))
    opt_state = optimizer.init(params)
    split_idx = math.floor(c_data.shape[0] * 0.9)
    train_ds, val_ds = c_data[:split_idx], c_data[split_idx:]
    num_complete_batches, leftover = divmod(train_ds.shape[0], batch_size)
    num_batches = num_complete_batches + bool(leftover)
    for step in range(training_steps):
        i = step % num_batches
        batch = jax.lax.dynamic_slice_in_dim(train_ds, i * batch_size, batch_size)
        params, opt_state = update(params, opt_state, batch)
        print(".", end="")

        if step % eval_frequency == 0:
            num_val_batches = val_ds.shape[0] // batch_size
            i = step % num_val_batches
            val_batch = jax.lax.dynamic_slice_in_dim(val_ds, i * batch_size, batch_size)
            output: AEOutput = model.apply(params, val_batch)
            loss = jnp.mean(jnp.square(val_batch - output.data))
            print("STEP: %5d; Validation loss: %.3f" % (step, loss))
    output: AEOutput = model.apply(params, c_data)
    jnp.save(f"10_models/{c}.model", params)
    np.save(f"10_models/{c}.latent", output.latent)

Training for chromosome chr10: 11009
.STEP:     0; Validation loss: 9.180
..........STEP:    10; Validation loss: 0.147
..........STEP:    20; Validation loss: 0.148
..........STEP:    30; Validation loss: 0.163
..........STEP:    40; Validation loss: 0.170
..........STEP:    50; Validation loss: 0.166
..........STEP:    60; Validation loss: 0.169
..........STEP:    70; Validation loss: 0.168
..........STEP:    80; Validation loss: 0.162
..........STEP:    90; Validation loss: 0.141
..........STEP:   100; Validation loss: 0.142
..........STEP:   110; Validation loss: 0.161
..........STEP:   120; Validation loss: 0.164
..........STEP:   130; Validation loss: 0.160
..........STEP:   140; Validation loss: 0.162
..........STEP:   150; Validation loss: 0.162
..........STEP:   160; Validation loss: 0.163
..........STEP:   170; Validation loss: 0.143
..........STEP:   180; Validation loss: 0.142
..........STEP:   190; Validation loss: 0.163
..........STEP:   200; Validation loss: 0.160
......